<a href="https://colab.research.google.com/github/daksha200/ML-Projects/blob/testrepo/Cryptocurrency_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 It is a digital medium of exchange that is encrypted and decentralized. Many people use cryptocurrencies as a form of investing because it gives great returns even in a short period.

Buying and selling result in a change in the price of any cryptocurrency, but buying and selling trends depend on many factors. Using machine learning for cryptocurrency price prediction can only work in situations where prices change due to historical prices that people see before buying and selling their cryptocurrency. So, in the section below, I will take you through how you can predict the bitcoin prices (which is one of the most popular cryptocurrencies) for the next 30 days.

I’ll start the task of Cryptocurrency price prediction by importing the necessary Python libraries and the dataset we need. For this task, I will collect the latest Bitcoin prices data from Yahoo Finance, using the yfinance API. This will help you collect the latest data each time you run this code:

In [3]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 6.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [7]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days= 730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD', start = start_date, end = end_date, progress = False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.head())

        Date         Open         High          Low        Close    Adj Close  \
0 2020-03-29  6245.624512  6250.467285  5920.085938  5922.042969  5922.042969   
1 2020-03-30  5925.538574  6517.195801  5903.234375  6429.841797  6429.841797   
2 2020-03-31  6430.606445  6504.515137  6374.162109  6438.644531  6438.644531   
3 2020-04-01  6437.319336  6612.573730  6202.373535  6606.776367  6606.776367   
4 2020-04-02  6606.776367  7088.247559  6595.918457  6793.624512  6793.624512   

        Volume  
0  28373690931  
1  37101651525  
2  32786468812  
3  40346426266  
4  47660646124  


In the above code, I have collected the latest data of Bitcoin prices for the past 730 days, and then I have prepared it for any data science task. Now, let’s have a look at the shape of this dataset to see if we are working with 730 rows or not:

In [9]:
data.shape

(731, 7)

So the dataset contains 731 rows, where the first row contains the names of each column. Now let’s visualize the change in bitcoin prices till today by using a candlestick chart:

In [11]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Bitcoin Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

The Close column in the dataset contains the values we need to predict. So, let’s have a look at the correlation of all the columns in the data concerning the Close column:

In [13]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    1.000000
High         0.998634
Low          0.998353
Open         0.996834
Volume       0.285698
Name: Close, dtype: float64


Predicting the future prices of cryptocurrency is based on the problem of Time series analysis. The AutoTS library in Python is one of the best libraries for time series analysis. So here I will be using the AutoTS library to predict the bitcoin prices for the next 30 days:

In [17]:
!pip install AutoTS

     |████████████████████████████████| 460 kB 4.1 MB/s 


In [21]:
from autots import AutoTS

model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast

print(forecast)

Inferred frequency is: D
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



Model Number: 6 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 7 with model DatepartRegression in generation 0 of 10
Epoch 1/50
22/22 [==============================] - 12s 10ms/step - loss: 0.4162
Epoch 2/50
22/22 [==============================] - 0s 9ms/step - loss: 0.4153
Epoch 3/50
22/22 [==============================] - 0s 10ms/step - loss: 0.4134
Epoch 4/50
22/22 [==============================] - 0s 14ms/step - loss: 0.4129
Epoch 5/50
22/22 [==============================] - 0s 12ms/step - loss: 0.4126
Epoch 6/50
22/22 [==============================] - 0s 14ms/step - loss: 0.4082
Epoch 7/50
22/22 [==============================] - 0s 12ms/step - loss: 0.4042
Epoch 8/50
22/22 [==============================] - 0s 11ms/step - loss: 0.4008
Epoch 9/50
22/22 [==============================] - 0s 11ms/step - loss: 0.3968
Epoch 10/50
22/22 [==============================] - 0s 10ms/step - loss: 0.3950
Epoch 11/50
22/22 [==============================] - 0s 14ms/step - loss: 0.3901
Epoch 12/50
22/22 [==============================]

...

126 - DatepartRegression with avg smape 15.22: 
Model Number: 127 of 133 with model DatepartRegression for Validation 2


[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


127 - DatepartRegression with avg smape 83.92: 
Model Number: 128 of 133 with model GLS for Validation 2
128 - GLS with avg smape 14.29: 
Model Number: 129 of 133 with model DatepartRegression for Validation 2
129 - DatepartRegression with avg smape 23.22: 
Model Number: 130 of 133 with model DatepartRegression for Validation 2
130 - DatepartRegression with avg smape 15.08: 
Model Number: 131 of 133 with model DatepartRegression for Validation 2
131 - DatepartRegression with avg smape 24.74: 
Model Number: 132 of 133 with model GLM for Validation 2
132 - GLM with avg smape 19.37: 
Model Number: 133 of 133 with model GLM for Validation 2
133 - GLM with avg smape 20.2: 


/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/links.py:190: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:889: RuntimeWarning:

divide by zero encountered in true_divide



                   Close
2022-03-30  43124.201579
2022-03-31  43197.924577
2022-04-01  43192.725000
2022-04-02  43149.630908
2022-04-03  43051.157780
2022-04-04  42918.736117
2022-04-05  42828.810107
2022-04-06  42769.810596
2022-04-07  42734.819206
2022-04-08  42723.151676
2022-04-09  42750.008626
2022-04-10  42818.738493
2022-04-11  42912.449235
2022-04-12  43013.122949
2022-04-13  43138.881689
2022-04-14  43276.327653
2022-04-15  43417.107389
2022-04-16  43543.560872
2022-04-17  43627.030452
2022-04-18  43708.751302
2022-04-19  43766.532259
2022-04-20  43825.600814
2022-04-21  43885.929183
2022-04-22  43953.385677
2022-04-23  44001.035661
2022-04-24  43978.474837
2022-04-25  43907.937012
2022-04-26  43854.029639
2022-04-27  43831.032878
2022-04-28  43826.002930


**SUMMARY**

Buying and selling result in a change in the price of any cryptocurrency, but buying and selling trends depend on many factors. Using machine learning for cryptocurrency price prediction can only work in situations where prices change due to historical prices that people see before buying and selling their cryptocurrency. 